In [1]:
#!pip3 uninstall carla
import glob
import os
import sys
import random
import time
import numpy as np
import cv2

In [1]:
!pip3 install --user setuptools

In [3]:
!pip3 install --user wheel

In [26]:
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

In [2]:
!pip3 install carla==0.9.13

     ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.7 MB 1.3 MB/s eta 0:00:04
      --------------------------------------- 0.1/4.7 MB 656.4 kB/s eta 0:00:08
      --------------------------------------- 0.1/4.7 MB 930.9 kB/s eta 0:00:05
     - -------------------------------------- 0.2/4.7 MB 1.1 MB/s eta 0:00:05
     -- ------------------------------------- 0.3/4.7 MB 1.0 MB/s eta 0:00:05
     --- ------------------------------------ 0.4/4.7 MB 1.3 MB/s eta 0:00:04
     --- ------------------------------------ 0.4/4.7 MB 1.3 MB/s eta 0:00:04
     ----- ---------------------------------- 0.6/4.7 MB 1.6 MB/s eta 0:00:03
     ------ --------------------------------- 0.7/4.7 MB 1.7 MB/s eta 0:00:03
     ------- -------------------------------- 0.9/4.7 MB 1.9 MB/s eta 0:00:03
     ---------- ----------------------------- 1.2/4.7 MB 2.4 MB/s eta 0:00:02
     ----------- ---------------------------- 1.4/4.7 MB 2.6 MB/s e

In [2]:
# Import the CARLA Python API library and some utils
import carla 
import math 
import random 
import time 

In [3]:
# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 

In [4]:
client.set_timeout(15.0) # seconds

In [5]:
client.get_client_version()

'0.9.13'

In [14]:
actor_list = []
IM_WIDTH = 640
IM_HEIGHT = 480

def process_img(image):
    i = np.array(image.raw_data)
    i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))
    i3 = i2[:, :, :3]
    cv2.imshow("", i3)
    cv2.waitKey(1)
    return i3/255.0

def print_gyro_data(meas):
    print("IMU-GYRO DATA : ", meas.gyroscope)

try:
    client = carla.Client('localhost', 2000)
    client.set_timeout(5.0)

    world = client.get_world()
    
    blueprint_library = world.get_blueprint_library()
    
    bp = blueprint_library.filter('model3')[0]
    spawn_point = random.choice(world.get_map().get_spawn_points())
    vehicle = world.spawn_actor(bp, spawn_point)
    
    # Move the spectator behind the vehicle 
    spectator = world.get_spectator() 
    transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
    spectator.set_transform(transform) 
    
    vehicle.apply_control(carla.VehicleControl(throttle=0.5, steer=0.0))
    actor_list.append(vehicle)
    
    # ------------------------------------------------------------------------------#
    # https://carla.readthedocs.io/en/latest/cameras_and_sensors
    # get the blueprint for this sensor
    blueprint = blueprint_library.find('sensor.camera.rgb')
    # change the dimensions of the image
    blueprint.set_attribute('image_size_x', f'{IM_WIDTH}')
    blueprint.set_attribute('image_size_y', f'{IM_HEIGHT}')
    blueprint.set_attribute('fov', '110')

    # Adjust sensor relative to vehicle
    spawn_point = carla.Transform(carla.Location(x=2.5, z=0.7))

    # spawn the sensor and attach to vehicle.
    sensor = world.spawn_actor(blueprint, spawn_point, attach_to=vehicle)

    # add sensor to list of actors
    actor_list.append(sensor)
    
    # do something with this sensor
    sensor.listen(lambda data: process_img(data))   
    
    # ---------------------------------------------------------------------------#
    
    # get the blueprint for IMU
    blueprint_imu = blueprint_library.find('sensor.other.imu')
    
    # Adjust sensor relative to vehicle
    spawn_point_imu = carla.Transform(carla.Location(x=3, z=0.5))

    # spawn the sensor and attach to vehicle.
    sensor_imu = world.spawn_actor(blueprint_imu, spawn_point_imu, attach_to=vehicle)

    # add sensor to list of actors
    actor_list.append(sensor_imu)
    
    sensor_imu.listen(lambda meas: print_gyro_data(meas))
    
    # ---------------------------------------------------------------------------#
    
    # get the blueprint for Lidar
    blueprint_lidar = blueprint_library.find('sensor.lidar.ray_cast')

    # Adjust sensor relative to vehicle
    spawn_point_lidar = carla.Transform(carla.Location(x=4, z=0.9))

    # spawn the sensor and attach to vehicle.
    sensor_lidar = world.spawn_actor(blueprint_lidar, spawn_point_lidar, attach_to=vehicle)

    # add sensor to list of actors
    actor_list.append(sensor_lidar)
    
    # do something with this sensor
    sensor_lidar.listen(lambda point_cloud: point_cloud.save_to_disk('lidar_output/%.6d.ply' % point_cloud.frame)) 
   
    
    time.sleep(30)


finally:

    print('destroying actors')
    for actor in actor_list:
        actor.destroy()
    print('done.')

IMU-GYRO DATA :  Vector3D(x=0.000000, y=0.000000, z=0.000000)
IMU-GYRO DATA :  Vector3D(x=0.000000, y=0.000000, z=0.000000)
IMU-GYRO DATA :  Vector3D(x=0.000003, y=-0.000003, z=-0.000000)
IMU-GYRO DATA :  Vector3D(x=0.000000, y=-0.000000, z=-0.000000)
IMU-GYRO DATA :  Vector3D(x=-0.000000, y=0.000001, z=0.000000)
IMU-GYRO DATA :  Vector3D(x=0.000000, y=0.000000, z=0.000000)
IMU-GYRO DATA :  Vector3D(x=0.000001, y=-0.000000, z=-0.000000)
IMU-GYRO DATA :  Vector3D(x=-0.000001, y=-0.008687, z=-0.000050)
IMU-GYRO DATA :  Vector3D(x=-0.000001, y=-0.007489, z=-0.000016)
IMU-GYRO DATA :  Vector3D(x=0.000004, y=0.000446, z=0.000173)
IMU-GYRO DATA :  Vector3D(x=0.000000, y=0.001941, z=0.000035)
IMU-GYRO DATA :  Vector3D(x=-0.000001, y=0.001673, z=-0.000003)
IMU-GYRO DATA :  Vector3D(x=0.000000, y=0.001162, z=-0.000003)
IMU-GYRO DATA :  Vector3D(x=-0.000001, y=0.000756, z=-0.000003)
IMU-GYRO DATA :  Vector3D(x=-0.000001, y=0.000486, z=-0.000003)
IMU-GYRO DATA :  Vector3D(x=-0.000395, y=-0.001908

IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000010, z=0.013811)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000010, z=0.013661)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000010, z=0.013515)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000009, z=0.013371)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000009, z=0.013231)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000009, z=0.013094)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000009, z=0.012959)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000009, z=0.012827)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000008, z=0.012698)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000008, z=0.012572)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000008, z=0.012449)
IMU-GYRO DATA :  Vector3D(x=-0.000003, y=0.000008, z=0.012328)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000008, z=0.012210)
IMU-GYRO DATA :  Vector3D(x=-0.000005, y=0.000008, z=0.012094)
IMU-GYRO DATA :  Vector3D(x=-0.000004, y=0.000007, z=0.011981)
IMU-GYRO DATA :  Vector3D(x=-0.000003, y=0.000007, z=0.